# TextRank: Bringing Order into Texts
In this part you will implement [TextRank: Bringing Order into Texts](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf) paper.

[Mihalcea](https://scholar.google.com/citations?user=UetM7FgAAAAJ&hl=en) and [Tarau](https://scholar.google.com/citations?user=JUMRc-oAAAAJ&hl=en) in this paper, introduced TextRank – a **graphbased ranking model for text processing**, and show how it can be successfully used for natural language applications. In particular, they proposed and evaluated two innovative unsupervised approaches for keyword and sentence extraction.

The basic idea implemented by a graph-based ranking model is that of “voting” or “recommendation”.

When one vertex links to another one, it is basically casting a vote for that other vertex. The higher the number of votes that are cast for a vertex, the higher the importance of the vertex. Moreover, the importance of the vertex casting the vote determines how important the vote itself is, and this information is also taken into account by the ranking model.**This paper relies on Google's PageRank**.

## Defenition

Formally, let $G=(V, E)$ be a directed graph with the set of vertices $V$ and set of edges $E$, where $E$ is a subset of $V \times V$. For a given vertex $V_i$ , let $In(V_i)$ be the set of vertices that point to it (predecessors), and let $Out(V_i)$ be the set of vertices that vertex $V_i$ points to (successors). The score of a vertex $V_i$ is defined as follows (Brin and Page, 1998):

$S(V_i) = (1-d) + d* \sum_{j \in In(V_i)} \frac{1}{|Out(V_j)|}S(V_j)$


where d is a damping factor and usually set to 0.85.

## Graph representation

TextRank builds a weighted graph representation of a document using words as nodes and **co-ocurrence** [<sup>1</sup>](#fn1) frequencies between pairs of words as edge weights. It then applies PageRank to this graph, and treats the PageRank score of each word as its significance.

<img src="textrank.png" width="400" align="center">

<span id="fn1"> [1]: In linguistics, co-occurrence or cooccurrence is an above-chance frequency of occurrence of two terms (also known as coincidence or concurrence) from a text corpus alongside each other in a certain order. For example, when the term "strong coffee" appears in a document, the term "espresso bean" probably also tends to occur in that document.</span>


In [8]:
import os
import sys
import copy
import collections
import numpy as np
import nltk
import nltk.tokenize

sys.path.append(".")

import pandas
import page_rank
import text_rank

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/alireza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alireza/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Preprocessing
**Tokenization** is a common task in **Natural Language Processing** (NLP). It’s a fundamental step in both traditional NLP methods like Count Vectorizer and Advanced Deep Learning-based architectures like [Transformers](https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/?utm_source=blog&utm_medium=what-is-tokenization-nlp).

This is the process by which a large quantity of text is divided into smaller parts called **tokens**.

Natural Language toolkit has very important module [**NLTK**](https://www.nltk.org/api/nltk.tokenize.html) tokenize sentences which further comprises of sub-modules word tokenize and sentence tokenize.

We use the method [word_tokenize()](https://www.geeksforgeeks.org/python-nltk-nltk-tokenizer-word_tokenize/) to split a sentence into words. Please refer to below word tokenize NLTK example to understand the theory better.
```python
Input: "I love Applied linear algebra! specially the projects."
Output: ['I', 'love', 'Applied', 'linear', 'algebra', '!', 'specially', 'the', 'projects', '.']
```
After tokenizing the document we should filter irrelevant [PoS tags](https://en.wikipedia.org/wiki/Part-of-speech_tagging) and punctuation (e.g, !, ?).

In [11]:
def __preprocess_document(document, relevant_pos_tags):
    '''
    This function accepts a string representation 
    of a document as input, and returns a tokenized
    list of words corresponding to that document.
    '''
        
    # Tokenizing the document
    words = nltk.word_tokenize(document) # Your code here
    # print(words)
    
    # PoS tagging
    # Your code here, use nltk.pos_tag for words and make a list of second pair
    temp1 = [nltk.pos_tag([sent]) for sent in words]
    pos_tags = [sent[0][1] for sent in temp1]

    # Filter out words with irrelevant POS tags
    filtered_words = []
    for index, word in enumerate(words):
        word = word.lower()
        tag = pos_tags[index]
        # TODO: append `word` to `filtered_words` if the word is not a punctuation and pos is relevant.
        # You can use `__is_punctuation` function and `relevant_pos_tags`
        if word in [".", "?", "!", ",", "\"", ":", ";", "'", "-"]: 
            continue
        if not tag in relevant_pos_tags:
            continue
        filtered_words.append(word)

    return filtered_words

In [13]:
file_name = "data/Cinderalla.txt"
file_path = os.path.join(os.path.abspath(''), file_name)
document = open(file_path).read()
document = text_rank.__ascii_only(document)
relevant_pos_tags=["NN", "ADJ"]

In [14]:
__preprocess_document(document, relevant_pos_tags)

['time',
 'gentleman',
 'beautiful',
 'kind',
 'wife',
 'proudest',
 'woman',
 'land',
 'marriage',
 'haughty',
 'mother',
 'gentleman',
 'daughter',
 'wife',
 'cinderella',
 'goodness',
 'sweetest',
 'kingdom',
 'cinderella',
 'stepmother',
 'beauty',
 'charm',
 'hardest',
 'dreadful',
 'work',
 'house',
 'cinderella',
 'scrubbed',
 'floor',
 'bed',
 'fancy',
 'fun',
 'dress-up',
 'son',
 'ball',
 'land',
 'attend',
 'cinderella',
 'step-mother',
 'talk',
 'nothing',
 'ball',
 'day',
 'sent',
 'kingdom',
 'cinderella',
 'help',
 'ball',
 'excellent',
 'taste',
 'advice',
 'eldest',
 'sister',
 'cinderella',
 'ball',
 'cinderella',
 'head',
 'nothing',
 'wear',
 'fit',
 'borrow',
 'something',
 'dirty',
 'cinderwench',
 'everyone',
 'laugh',
 'sight',
 'cinderwench',
 'day',
 'cinderella',
 'step-mother',
 'help',
 'burst',
 'enter',
 'beautiful',
 'ball',
 'wept',
 'cinderella',
 'fairy',
 'godmother',
 'cinderella',
 'wish',
 'attend',
 'ball',
 'cinderella',
 'sobs',
 'fairy',
 'god

## Ranking
In this section, first we will implement weighted PageRank and use this function to implement textRank.

### PageRank using Power method

In [108]:
def power_iteration(transition_weights, rsp=0.15, epsilon=0.00001, max_iterations=1000):
    # Clerical work:
    transition_weights = pandas.DataFrame(transition_weights)
    nodes = page_rank.__extract_nodes(transition_weights)
    transition_weights = page_rank.__make_square(transition_weights, nodes, default=0.0)
    transition_weights = page_rank.__ensure_rows_positive(transition_weights)

    # Setup:
    state = page_rank.__start_state(nodes)
    transition_probabilities = page_rank.__normalize_rows(transition_weights)
    
    # Compute transition matrix
    # Your code here
    alpha = 1.0 / float(len(nodes)) * rsp
    transition_probabilities = transition_probabilities.copy().multiply(1.0 - rsp) + alpha
    
    # ---------- Power Method ------------- #
    # Power iteration:
    # TODO: implement power method
    # Use state.copy() for copying to old_state
    for iteration in range(max_iterations):
        state_last = state.copy()
        state = state @ transition_probabilities
        check = np.abs(state - state_last)
        # print(check)
        if np.sum(np.array(check))/len(nodes) < epsilon:
            return state      
    return state

### TextRank algorithm
Authors used a co-occurrence relation (as we discussed), controlled by the distance between word occurrences: **two vertices are connected** if their corresponding lexical units co-occur within a **window of maximum  words**, where  can be set anywhere from 2 to 10 word.


The vertices added to the graph can be restricted with **syntactic filters**, which select only lexical units of a certain part of speech. One can for instance consider only nouns and verbs for addition to the graph, and consequently draw potential edges based only on relations that can be established between nouns and verb. Experiments showed that **best results observed for nouns ("NN") and adjectives ("ADJ") only**.

In [102]:
def textrank(document, window_size=2, rsp=0.15, relevant_pos_tags=["NN", "ADJ"]):
    '''
    Accepts a string representation
    of a document and returns Pandas matrix that maps words to their related TextRank scores.
    Keyword arguments:
    window_size: window of maximum words, can be set between 2 to 10. (default 2)
    rsp:
    relevant_pos_tags: list tags that graph is restricted by (default ["NN", "ADJ"])
    '''
    
    # Tokenize document:
    words = __preprocess_document(document, relevant_pos_tags)
    
    
    # Building the weighted graph:
    # nodes: words
    # edge weights number of times words cooccur within a window of predetermined size
    edge_weights = collections.defaultdict(lambda: collections.Counter())
    for index, word in enumerate(words):
        for other_index in range(index - window_size, index + window_size + 1):
            if other_index >= 0 and other_index < len(words) and other_index != index:
                other_word = words[other_index]
                edge_weights[word][other_word] += 1.0

    # Apply `power_iteration` to `edge_weights` and sort the output
    # Your code here
    
    word_probabilities = power_iteration(edge_weights)
    # temp3 = power_iteration(edge_weights)
    # word_probabilities = sorted(temp3)
    
    return word_probabilities

## Apply TextRank

In [17]:
def apply_text_rank(file_name, title="a document"):
    print("Reading \"%s\" ..." % title)
    # Opening:
    
    file_path = os.path.join(os.path.abspath(''), file_name)
    document = open(file_path).read()
    document = text_rank.__ascii_only(document)
    
    print("Applying TextRank to \"%s\" ..." % title)
    
    # TODO: get TextRank vector
    keyword_scores = textrank(document) # Call `textrank` function here with arbitrary inputs

    print()
    header = "Keyword Significance Scores for \"%s\":" % title
    print(header)
    print("-" * len(header))
    print(keyword_scores)
    print()

### Cinderalla story

In [107]:
apply_text_rank("data/Cinderalla.txt", "Cinderalla")

Reading "Cinderalla" ...
Applying TextRank to "Cinderalla" ...
advice       0.001673
anything     0.003811
approach     0.004109
attend       0.000810
attention    0.003161
               ...   
wife         0.002127
wish         0.005741
woman        0.001580
work         0.001617
world        0.002231
Length: 127, dtype: float64
time         0.001625
gentleman    0.000804
beautiful    0.001645
kind         0.000323
wife         0.001042
               ...   
awful        0.000715
way          0.000925
forgave      0.000157
heart        0.001551
palace       0.001753
Length: 127, dtype: float64
time         0.000318
gentleman    0.000169
beautiful    0.000181
kind         0.000163
wife         0.000147
               ...   
awful        0.000683
way          0.000229
forgave      0.000123
heart        0.000349
palace       0.000292
Length: 127, dtype: float64
time         0.000233
gentleman    0.000176
beautiful    0.000425
kind         0.000014
wife         0.000161
               ..

### Beauty_and_the_Beast story

In [99]:
apply_text_rank("data/Beauty_and_the_Beast.txt", "Beauty and the Beast")

Reading "Beauty and the Beast" ...
Applying TextRank to "Beauty and the Beast" ...

Keyword Significance Scores for "Beauty and the Beast":
-------------------------------------------------------
time       0.012105
king       0.044680
queen      0.045831
sad        0.017342
day        0.007060
             ...   
royal      0.008437
wedding    0.010905
bride      0.011208
wore       0.012153
rode       0.006171
Length: 75, dtype: float64



In [9]:
# Optional: test textRank on another documents :))